In [ ]:
# !pip install matplotlib

#import the required packages
import os
import time
from tensorflow import keras
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from IPython import display
from keras.preprocessing.image import ImageDataGenerator


In [ ]:
gpus = tf.config.list_physical_devices('GPU')
gpus

if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

# +
# os.environ['CUDA_VISIBLE_DEVICES'] = '0'
# -

In [ ]:
img_height, img_width = 64, 64
batch_size = 51

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  './MiriSegal',
  image_size=(img_height, img_width),
  batch_size=batch_size,
  label_mode=None)

plt.figure(figsize=(10, 10))
for images in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.axis("off")

for image_batch in train_ds:
  print(image_batch.shape)
  break

tf.data.experimental.AUTOTUNE

AUTOTUNE = tf.data.experimental.AUTOTUNE

# +
# train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
# -

normalization_layer = layers.experimental.preprocessing.Rescaling(scale= 1./127.5, offset=-1)

normalized_ds = train_ds.map(lambda x: normalization_layer(x))
image_batch = next(iter(normalized_ds))
first_image = image_batch[0]
print(np.min(first_image), np.max(first_image)) 

In [ ]:
latent_dim = 100
noise_dim = (1,1,latent_dim)

In [ ]:
def generator_model():
    
    inputs = keras.Input(shape=noise_dim, name='input_layer')
    x = layers.Conv2DTranspose(64 * 8, kernel_size=4, strides= 4, padding='same', kernel_initializer=tf.keras.initializers.RandomNormal(
    mean=0.0, stddev=0.02), use_bias=False, name='conv_transpose_1')(inputs)
    x = layers.BatchNormalization(momentum=0.1,  epsilon=0.8, center=1.0, scale=0.02, name='bn_1')(x)
    x = layers.ReLU(name='relu_1')(x)
    
    x = layers.Conv2DTranspose(64 * 4, kernel_size=4, strides= 2, padding='same', kernel_initializer=tf.keras.initializers.RandomNormal(
    mean=0.0, stddev=0.02), use_bias=False, name='conv_transpose_2')(x)
    x = layers.BatchNormalization(momentum=0.1,  epsilon=0.8, center=1.0, scale=0.02, name='bn_2')(x)
    x = layers.ReLU(name='relu_2')(x)
    
    x = layers.Conv2DTranspose(64 * 2, 4, 2, padding='same', kernel_initializer=tf.keras.initializers.RandomNormal(
    mean=0.0, stddev=0.02), use_bias=False, name='conv_transpose_3')(x)
    x = layers.BatchNormalization(momentum=0.1,  epsilon=0.8,  center=1.0, scale=0.02, name='bn_3')(x)
    x = layers.ReLU(name='relu_3')(x)
  

    x = layers.Conv2DTranspose(64 * 1, 4, 2, padding='same', kernel_initializer=tf.keras.initializers.RandomNormal(
    mean=0.0, stddev=0.02), use_bias=False, name='conv_transpose_4')(x)
    x = layers.BatchNormalization(momentum=0.1,  epsilon=0.8,  center=1.0, scale=0.02, name='bn_4')(x)
    x = layers.ReLU(name='relu_4')(x)
    
    outputs = layers.Conv2DTranspose(3, 4, 2,padding='same', kernel_initializer=tf.keras.initializers.RandomNormal(
    mean=0.0, stddev=0.02), use_bias=False, activation='tanh', name='conv_transpose_5')(x)
   
    model = tf.keras.Model(inputs, outputs, name="Generator")
    return model

In [ ]:
generator = generator_model()
generator.save('dcgan_gen.h5')
generator.summary()

In [ ]:
def discriminator_model():
    
    inputs = keras.Input(shape=(64, 64, 3), name='input_layer')
    x = layers.Conv2D(64, kernel_size=4, strides= 2, padding='same', kernel_initializer=tf.keras.initializers.RandomNormal(
    mean=0.0, stddev=0.02), use_bias=False, name='conv_1')(inputs)
    x = layers.LeakyReLU(0.2, name='leaky_relu_1')(x)
    
    x = layers.Conv2D(64 * 2, kernel_size=4, strides= 2, padding='same', kernel_initializer=tf.keras.initializers.RandomNormal(
    mean=0.0, stddev=0.02), use_bias=False, name='conv_2')(x)
    x = layers.BatchNormalization(momentum=0.1,  epsilon=0.8, center=1.0, scale=0.02, name='bn_1')(x)
    x = layers.LeakyReLU(0.2, name='leaky_relu_2')(x)
    
    x = layers.Conv2D(64 * 4, 4, 2, padding='same', kernel_initializer=tf.keras.initializers.RandomNormal(
    mean=0.0, stddev=0.02), use_bias=False, name='conv_3')(x)
    x = layers.BatchNormalization(momentum=0.1,  epsilon=0.8, center=1.0, scale=0.02, name='bn_2')(x)
    x = layers.LeakyReLU(0.2, name='leaky_relu_3')(x)
  

    x = layers.Conv2D(64 * 8, 4, 2, padding='same', kernel_initializer=tf.keras.initializers.RandomNormal(
    mean=0.0, stddev=0.02), use_bias=False, name='conv_4')(x)
    x = layers.BatchNormalization(momentum=0.1,  epsilon=0.8, center=1.0, scale=0.02, name='bn_3')(x)
    x = layers.LeakyReLU(0.2, name='leaky_relu_4')(x)
    
    outputs = layers.Conv2D(1, 4, 4,padding='same', kernel_initializer=tf.keras.initializers.RandomNormal(
    mean=0.0, stddev=0.02), use_bias=False, activation='sigmoid', name='conv_5')(x)
    
    outputs = layers.Flatten()(outputs)
   
    model = tf.keras.Model(inputs, outputs, name="Discriminator")
    return model

In [ ]:
discriminator = discriminator_model()
discriminator.save('dcgan_disc.h5')
discriminator.summary()

In [ ]:
binary_cross_entropy = tf.keras.losses.BinaryCrossentropy()

def generator_loss(label, fake_output):
    gen_loss = binary_cross_entropy(label, fake_output)
    #print(gen_loss)
    return gen_loss

def discriminator_loss(label, output):
    disc_loss = binary_cross_entropy(label, output)
    #print(total_loss)
    return disc_loss

learning_rate = 0.0002 
generator_optimizer = tf.keras.optimizers.Adam(lr = 0.0002, beta_1 = 0.5, beta_2 = 0.999 )
discriminator_optimizer = tf.keras.optimizers.Adam(lr = 0.0002, beta_1 = 0.5, beta_2 = 0.999 )

In [ ]:
num_examples_to_generate = 25
# We will reuse this seed overtime to visualize progress
seed = tf.random.normal([num_examples_to_generate, 1, 1, latent_dim])

In [ ]:
# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
# @tf.function
def train_step(images, epoch):
    # noise vector sampled from normal distribution
    noise = tf.random.normal([BATCH_SIZE, 1, 1, latent_dim])

    real_output = discriminator(images, training=True)
    perf_real = real_output.numpy().mean()
    generated_images = generator(noise, training=True)
    fake_output = discriminator(generated_images, training=True)
    perf_fake = fake_output.numpy().mean()
    if perf_real > 0.8 and perf_fake < 0.4:
        print('SKIPPING discriminator training, real data: %f, fake data: %f' % (perf_real, perf_fake))
        skip_next = True
    else:
        print('Trying discriminator training, real data: %f, fake data: %f' % (perf_real, perf_fake))
        skip_next = False

    # we train discriminator only every 10th step to prevent it from being too strong
    if not skip_next:
        # Train Discriminator with real labels
        with tf.GradientTape() as disc_tape1:
            real_output = discriminator(images, training=True)
            perf_real = real_output.numpy().mean()
            print('real data: ',perf_real)
            real_targets = tf.ones_like(real_output)
            disc_loss1 = discriminator_loss(real_targets, real_output)
        # gradient calculation for discriminator for real labels    
        gradients_of_disc1 = disc_tape1.gradient(disc_loss1, discriminator.trainable_variables)

        # parameters optimization for discriminator for real labels   
        discriminator_optimizer.apply_gradients(zip(gradients_of_disc1,\
        discriminator.trainable_variables))

        # Train Discriminator with fake labels
        with tf.GradientTape() as disc_tape2:
            fake_output = discriminator(generated_images, training=True)
            fake_targets = tf.zeros_like(fake_output)
            disc_loss2 = discriminator_loss(fake_targets, fake_output)
            perf_fake = fake_output.numpy().mean()
            print('fake data: ',perf_fake)
        # gradient calculation for discriminator for fake labels 
        gradients_of_disc2 = disc_tape2.gradient(disc_loss2, discriminator.trainable_variables)

        # parameters optimization for discriminator for fake labels        
        discriminator_optimizer.apply_gradients(zip(gradients_of_disc2,\
        discriminator.trainable_variables))
    
    # Train Generator with real labels
    with tf.GradientTape() as gen_tape:
        generated_images = generator(noise, training=True)
        fake_output = discriminator(generated_images, training=True)
        real_targets = tf.ones_like(fake_output)
        gen_loss = generator_loss(real_targets, fake_output)

    # gradient calculation for generator for real labels     
    gradients_of_gen = gen_tape.gradient(gen_loss, generator.trainable_variables)
    
    # parameters optimization for generator for real labels
    generator_optimizer.apply_gradients(zip(gradients_of_gen,\
    generator.trainable_variables))    

In [ ]:
def train(dataset, epochs):
    for epoch in range(epochs):
        start = time.time()
        i = 0
        D_loss_list, G_loss_list = [], []
        for image_batch in dataset:
            i += 1
            skip_next = train_step(image_batch, epoch)
        print(epoch)
        if epoch % 50 ==0:
            display.clear_output(wait=True)
            generate_and_save_images(generator,
                                  epoch + 1,
                                  seed)
            print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

        # Save the model every 15 epochs
        if (epoch + 1) % 500 == 0:
            generator.save_weights('training_weights/gen_'+ str(epoch)+'.h5')
            discriminator.save_weights('training_weights/disc_'+ str(epoch)+'.h5')    

    # Generate after the final epoch
#     display.clear_output(wait=True)
    generate_and_save_images(generator,
                            epochs,
                            seed)

In [ ]:
def generate_and_save_images(model, epoch, test_input):
  # Notice `training` is set to False.
  # This is so all layers run in inference mode (batchnorm).
    predictions = model(test_input, training=False)
    print(predictions.shape)
    fig = plt.figure(figsize=(4,4))

    for i in range(predictions.shape[0]):
        plt.subplot(5, 5, i+1)
        pred = (predictions[i, :, :, :] + 1 ) * 127.5
        pred = np.array(pred)  
        plt.imshow(pred.astype(np.uint8))
        plt.axis('off')
    plt.savefig('images/image_at_epoch_{:d}.png'.format(epoch))
    plt.show()

In [ ]:
train(normalized_ds, 10000)

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.1,
    width_shift_range=0.05,
    height_shift_range=0.05,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode='constant', cval=1.0)
for a in normalized_ds.take(-1):
    pass
dg = datagen.flow(a,batch_size=128)

In [ ]:
def train2(dataset, epochs):
    for epoch in range(epochs):
        start = time.time()
        i = 0
        D_loss_list, G_loss_list = [], []
        for ii in range(3):
            image_batch = dataset.next()
            i += 1
            skip_next = train_step(image_batch, epoch)
        print(epoch)
        if epoch % 50 ==0:
            display.clear_output(wait=True)
            generate_and_save_images(generator,
                                  epoch + 1,
                                  seed)
            print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

        # Save the model every 50 epochs
        if (epoch + 1) % 50 == 0:
            generator.save_weights('training_weights/gen_'+ str(epoch)+'.h5')
            discriminator.save_weights('training_weights/disc_'+ str(epoch)+'.h5')    

    # Generate after the final epoch
#     display.clear_output(wait=True)
    generate_and_save_images(generator,
                            epochs,
                            seed)

In [ ]:
train2(dg, 10000)